In [ ]:
#import data/data.hdf5
import h5py
import numpy as np
import os
from sklearn.model_selection import train_test_split
DATA_DIR = "data"
DATASET_FILE_NAME = "data.hdf5"
def load_dataset():
    data_file = os.path.join(DATA_DIR, DATASET_FILE_NAME)
    hf = h5py.File(data_file, 'r')
    X_grids = np.array(hf.get('X_grids'))
    X_scalars = np.array(hf.get('X_scalars'))
    Y = np.array(hf.get('Y'))

    return X_grids, X_scalars, Y

dataset = load_dataset()

'''
grid 20x10x? vector
heldBlock 8 vector 1-hot
activeBlock 7 vector 1-hot
blockPos 2 vector
blockRot 4 vector 1-hot
nextMove 10 vector 1-hot
'''

In [ ]:
#reshape X_grids into 3-D array where each entry is a 20x10 grid
def reshape_X_grids(X_grids):
    X_grids_reshaped = np.zeros((X_grids.shape[0], 20, 10))
    for i in range(X_grids.shape[0]):
        X_grids_reshaped[i] = X_grids[i].reshape(20, 10)
    return X_grids_reshaped

dataset[0] = reshape_X_grids(dataset[0])

In [ ]:
#split dataset into train and test set by random
def split_dataset(X_grids, X_scalars, Y):
    X_grids_train, X_grids_test, X_scalars_train, X_scalars_test, Y_train, Y_test = train_test_split(X_grids, X_scalars, Y, test_size=0.2, random_state=42)
    return ((X_grids_train, X_scalars_train, Y_train), (X_grids_test, X_scalars_test, Y_test))

train_dataset, test_dataset = split_dataset(X_grids, X_scalars, Y)
print(test_dataset[2].shape)

In [ ]:
# Convert currentBlock scalar into N-hot vector
def scalarToVector(scalar, size):
    vector = np.zeros(size)
    vector[scalar] = 1

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, concatenate
from tensorflow.keras.models import Model

# Define input layers for each type of input
cnn_input = Input(shape=(20, 10, 1))
heldBlock_input = Input(shape=(8,))
activeBlock_input = Input(shape=(7,))
rotation_input = Input(shape=(4,))
position_input = Input(shape=(2,))

# CNN branch
x = Conv2D(32, (3, 3), activation='relu')(cnn_input)
x = Flatten()(x)

# Branch for 8-vector input
x2 = Dense(16, activation='relu')(eight_vector_input)

# Branch for 7-vector input
x3 = Dense(16, activation='relu')(seven_vector_input)

# Branch for 4-vector input
x4 = Dense(16, activation='relu')(four_vector_input)

# Branch for 2-vector input
x5 = Dense(16, activation='relu')(two_vector_input)

# Concatenate all the branches
combined = concatenate([x, x2, x3, x4, x5])

# Add more layers if needed
combined = Dense(64, activation='relu')(combined)
output = Dense(10, activation='softmax')(combined)  # Replace 'output_dim' with the number of output classes or neurons

# Create the model
model = Model(inputs=[cnn_input, eight_vector_input, seven_vector_input, four_vector_input, two_vector_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
#model.fit([cnn_data, eight_vector_data, seven_vector_data, four_vector_data, two_vector_data], labels, epochs=epochs, batch_size=batch_size)

def create_model():
    pass
    


